# WARNING
**Please make sure to "COPY AND EDIT NOTEBOOK" to use compatible library dependencies! DO NOT CREATE A NEW NOTEBOOK AND COPY+PASTE THE CODE - this will use latest Kaggle dependencies at the time you do that, and the code will need to be modified to make it work. Also make sure internet connectivity is enabled on your notebook**

# Preliminaries
Write requirements to file, anytime you run it, in case you have to go back and recover dependencies. **MOST OF THESE REQUIREMENTS WOULD NOT BE NECESSARY FOR LOCAL INSTALLATION**

Requirements are hosted for each notebook in the companion github repo, and can be pulled down and installed here if needed. Companion github repo is located at https://github.com/azunre/transfer-learning-for-nlp

In [1]:
!pip freeze > kaggle_image_requirements.txt

# Open Ended Text Generation with GPT-2

In [2]:
# Pipeline uses `gpt2` by default, but we specify it explicitly to be fully transparent
from transformers import pipeline
gpt = pipeline('text-generation',model='gpt2')

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


Now, let's generate some text with GPT-2

In [3]:
gpt("Transfer learning is a field of study", max_length=100)

[{'generated_text': "Transfer learning is a field of study that combines the field with the community's knowledge. We train new students to recognize, manage, and respond to the many problems facing this special role in their daily lives: interpersonal issues, physical and mental pain, and general health. Teaching students to interact with and understand each other is also an important part of our profession. As part of our education and community outreach work, we have launched programs to help prepare young adults to become leaders in a fast-paced and interactive"}]

A nonexhaustive list of other model choices suitable for text generation within the transformers library include "ctrl" (CTRL - huge! too big for Kaggle), "xlnet-base-cased" (XLNet), "transfo-xl-wt103" (Transformer XL)... These often need to be padded very carefully to work well, GPT-2 is the safest choice for open-ended text generation. See https://huggingface.co/transformers/usage.html#text-generation for more.

# Conversational Text Generation with DialoGPT

DialoGPT is an extension of GPT to conversational response generation

In [4]:
from transformers import AutoModelWithLMHead, AutoTokenizer # you can use these utility classes that automatically load the right classes
from transformers import GPT2LMHeadModel, GPT2Tokenizer # or these more specific classes directly
import torch

tokenizer = GPT2Tokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = GPT2LMHeadModel.from_pretrained("microsoft/DialoGPT-medium")

In [5]:
# Chat for 5 Lines
conversation_length = 5
for step in range(conversation_length):
    # encode new user input, add end-of-sentence token, return tensor
    new_user_inputs_ids = tokenizer.encode(input("User: ") + tokenizer.eos_token, return_tensors='pt')
    
    # add new input to chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_inputs_ids], dim=1) if step > 0 else new_user_inputs_ids
    
    # generate a response of up to max_length tokens
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    
    # display response
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

User:  hi robot


DialoGPT: Hello, human.


User:  huh?


DialoGPT: I'm a bot.


User:  ok, what is your name?


DialoGPT: Robot. I'm a robot.


User:  ok! Can you say something else?


DialoGPT: Robot. I'm a robot.


User:  Do you have children?


DialoGPT: Robot. I'm a robot.


**Because the notebook is not saving input text, here is the entire conversation so you know how we prompted the model**

User:  hi robot

DialoGPT: Hello, human.

User:  huh?

DialoGPT: I'm a bot.

User:  ok, what is your name?

DialoGPT: Robot. I'm a robot.

User:  ok! Can you say something else?

DialoGPT: Robot. I'm a robot.

User:  Do you have children?

DialoGPT: Robot. I'm a robot.